# PyMoq

This is a recreational attempt at replicating parts of the c# moq library.

## Faking attributes

### Reading attributes

Listing all attributes is easy:

In [30]:
dir(IWeb)

['__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_protocol',
 '_is_runtime_protocol',
 'get']

These have to be filtered, we only need the 'public' attributes. By convention, these are the ones not starting with an underscore.

In [85]:
from typing import Any, List

def is_public(name: str) -> bool:
    return not name.startswith('_')

def get_public_attributes(cls : Any) -> List[str]:
    return list(filter(is_public, dir(cls)))

In [86]:
get_public_attributes(IWeb)

['get']

### Get Function by name

The string name of the functions from above can easily be used to access the corresponding function objects of a class:

In [84]:
getattr(IWeb, 'get')

<function __main__.IWeb.get(url: str) -> str>

### Signature validation

The inspect module gives easy access to the signature of a callable:

In [83]:
from inspect import signature

sig = signature(IWeb.get)
sig

<Signature (url: str) -> str>

We can also check if a given argument list is enough to call a function with the given signature:

In [73]:
sig.bind('url')

<BoundArguments (url='url')>

In [75]:
try:
    sig.bind()
except TypeError as e:
    print(e)

missing a required argument: 'url'


From this, we can create a wrapper that checks for the validity of a call, then does some optional pattern matching on the given arguments.

In [154]:
from typing import Callable

class FunctionMocker:
    def __init__(self, target_function: Callable[[Any], Any], top_level: str=""):
        self.name = target_function.__name__
        self.target_signature = signature(target_function)
        self._error_prefix = f'{top_level}.' if top_level else ''
        
    def _validate_signature(self, *args, **kwargs):
        try:
            self.target_signature.bind(*args, **kwargs)
        except TypeError as e:
            raise TypeError(f'{self._error_prefix}{self.name}{self.target_signature}: {e}')
        
    def __call__(self, *args, **kwargs):
        self._validate_signature(*args, **kwargs)
        
        print('Correct signature, start pattern matching')
        
    def __str__(self):
        return f'Moq: {self.name}{self.target_signature}'
    
    def __repr__(self):
        return str(self)

In [155]:
f = FunctionMocker(IWeb.get)
f('url')

Correct signature, start pattern matching


In [156]:
try:
    f()
except TypeError as e:
    print(e)

get(url: str) -> str: missing a required argument: 'url'


### A first simple Mocker

With the `FunctionMocker` from above we can implement a very simple version of a class-specific Mocker.

In [160]:
class Moq:
    def __init__(self, cls: Any):
        self.cls_name = cls.__name__
        self._extract_attributes(cls)
        
    def _extract_attributes(self, cls):
        self.attribute_names = get_public_attributes(cls)
        self.moq_functions = {name: FunctionMocker(getattr(cls, name), self.cls_name) for name in self.attribute_names}
        
    def __getattr__(self, name):
        if name not in self.moq_functions:
            raise AttributeError(f"No attribute with name '{name}' in class {self.cls_name}")
            
        return self.moq_functions[name]
    
    def __dir__(self):
        return dir(super()) + self.attribute_names

In [161]:
m = Moq(IWeb)

In [162]:
m.get()

TypeError: IWeb.get(url: str) -> str: missing a required argument: 'url'

In [163]:
try:
    m.not_found
except AttributeError as e:
    print(e)

No attribute with name 'not_found' in class IWeb
